In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, DenseGCNConv, DenseGraphConv, global_mean_pool, ASAPooling, GATv2Conv, GCNConv
from torch_geometric.utils import add_self_loops, degree
import numpy as np
import torch

In [40]:
class GraphConv(MessagePassing):
    def __init__(self, num_features, hidden_channels):
        super(GraphConv, self).__init__(aggr='mean')
        self.lin1 = nn.Linear(num_features, hidden_channels)
        self.lin2 = nn.Linear(hidden_channels, hidden_channels)
        self.pool = ASAPooling(hidden_channels, dropout=0.3)

    def forward(self, x,  edge_index, batch=None):
        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        x = F.relu(x)

        # Step 3: Start propagating messages.
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        x = self.propagate(edge_index, x=x, norm=norm)

        # Step 4: Return output.
        x = self.pool(x)
        # x = F.relu(x)
        # x = global_mean_pool(x, batch) # global mean pooling
        return x

    def message(self, x_j, norm):
        # Normalize node features.
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        # No additional update.
        return aggr_out

In [8]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(4, 160)
        self.conv2 = GCNConv(160, 6)

    def forward(self, data, batch=None):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)

        return F.log_softmax(x, dim=1)


In [7]:
train_loader = torch.load( '../export/train_loader.pt')
test_loader = torch.load('../export/test_loader.pt')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    for graph in train_loader:
        optimizer.zero_grad()
        out = model(graph)
        y = graph.y.type(torch.LongTensor)
        loss = F.nll_loss(out.view(-1), y)
        loss.backward()
        optimizer.step()


In [5]:
train_loader[0]

TypeError: 'DataLoader' object is not subscriptable

In [9]:
model = GCN()

In [17]:
gr = next(iter(train_loader))

model(gr).view(-1)

tensor([-1.7249, -1.7558, -1.7616, -1.7745, -1.9592, -1.7915],
       grad_fn=<ViewBackward0>)

In [16]:
gr.y.shape

torch.Size([6])

In [50]:
with torch.autocast('cuda'):
    out = gc(gr.x.float(), gr.edge_index)

In [51]:
out.shape

torch.Size([14833, 100])

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output, _, _ = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        loss_all += data.y.size(0) * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)


@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0

    for data in loader:
        data = data.to(device)
        pred = model(data.x, data.adj, data.mask)[0].max(dim=1)[1]
        correct += pred.eq(data.y.view(-1)).sum().item()
    return correct / len(loader.dataset)



In [10]:
gcn1 = DenseGCNConv(42225, 4)
grc = DenseGraphConv(42225, 4)

In [11]:
gcn1(data.x, data.edge_index)

RuntimeError: The size of tensor a (252532) must match the size of tensor b (2) at non-singleton dimension 2

In [44]:
Net(data.x, data.edge_index)

tensor([[0.1374]], grad_fn=<MeanBackward1>)

In [18]:
gcn1(data.x, data.edge_index)

RuntimeError: The size of tensor a (252532) must match the size of tensor b (2) at non-singleton dimension 2

In [ ]:

best_val_acc = test_acc = 0
for epoch in range(1, 151):
    train_loss = train(epoch)
    val_acc = test(val_loader)
    if val_acc > best_val_acc:
        test_acc = test(test_loader)
        best_val_acc = val_acc
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, '
          f'Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')